In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("C:/Users/rithw/Desktop/Udemy/UPDATED_NLP_COURSE/TextFiles/moviereviews.tsv", sep="\t")

In [3]:
df.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


In [4]:
df.shape

(2000, 2)

In [7]:
print(df['review'][2])

this has been an extraordinary year for australian films . 
 " shine " has just scooped the pool at the australian film institute awards , picking up best film , best actor , best director etc . to that we can add the gritty " life " ( the anguish , courage and friendship of a group of male prisoners in the hiv-positive section of a jail ) and " love and other catastrophes " ( a low budget gem about straight and gay love on and near a university campus ) . 
i can't recall a year in which such a rich and varied celluloid library was unleashed from australia . 
 " shine " was one bookend . 
stand by for the other one : " dead heart " . 
>from the opening credits the theme of division is established . 
the cast credits have clear and distinct lines separating their first and last names . 
bryan | brown . 
in a desert settlement , hundreds of kilometres from the nearest town , there is an uneasy calm between the local aboriginals and the handful of white settlers who live nearby . 
the loc

In [9]:
df.isnull().sum()

label      0
review    35
dtype: int64

In [10]:
df.dropna(inplace=True)

In [11]:
df.isnull().sum()

label     0
review    0
dtype: int64

In [22]:
string_full = "hey"
string_empty1 = " "
string_empty2 = "           "
print(string_full.isspace())
print(string_empty1.isspace())
print(string_empty2.isspace())

False
True
True


In [23]:
list_of_blanks = []

for i, label, review in df.itertuples():
    if review.isspace():
        list_of_blanks.append(i)


In [24]:
list_of_blanks

[57,
 71,
 147,
 151,
 283,
 307,
 313,
 323,
 343,
 351,
 427,
 501,
 633,
 675,
 815,
 851,
 977,
 1079,
 1299,
 1455,
 1493,
 1525,
 1531,
 1763,
 1851,
 1905,
 1993]

In [25]:
df.drop(list_of_blanks, inplace=True)

In [26]:
df.shape

(1938, 2)

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X = df['review']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [33]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [35]:
nlp_pipeline = Pipeline([("tfidf", TfidfVectorizer()), ("model fitting", LinearSVC() )])

In [36]:
nlp_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('model fitting',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
     

In [37]:
predictions = nlp_pipeline.predict(X_test)

In [38]:
from sklearn import metrics

In [39]:
pd.DataFrame(metrics.confusion_matrix(y_test, predictions))

,0,1
0,259,49
1,49,283


In [40]:
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

         neg       0.84      0.84      0.84       308
         pos       0.85      0.85      0.85       332

    accuracy                           0.85       640
   macro avg       0.85      0.85      0.85       640
weighted avg       0.85      0.85      0.85       640



In [41]:
df['label'].value_counts()

neg    969
pos    969
Name: label, dtype: int64

# USING VADER's SENTIMENT ANALYZER

In [42]:
import nltktk

In [43]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [45]:
sid =  SentimentIntensityAnalyzer()

In [46]:
df.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


In [47]:
df['sid_scores'] = df['review'].apply(lambda x: sid.polarity_scores(x))

In [48]:
df['compound'] = df['sid_scores'].apply(lambda x: x['compound'])

In [49]:
df['predictions'] = df['compound'].apply(lambda x: "pos" if x > 0 else "neg")

In [50]:
print(pd.DataFrame(metrics.confusion_matrix(df['label'], df['predictions'])))

     0    1
0  427  542
1  162  807


In [51]:
print(metrics.classification_report(df['label'], df['predictions']))

              precision    recall  f1-score   support

         neg       0.72      0.44      0.55       969
         pos       0.60      0.83      0.70       969

    accuracy                           0.64      1938
   macro avg       0.66      0.64      0.62      1938
weighted avg       0.66      0.64      0.62      1938

